In [1]:
import cv2
import numpy as np
import mediapipe as mp
import autopy

ModuleNotFoundError: No module named 'autopy'

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

# wScr, hScr = autopy.screen.size()
pX, pY = 0, 0
cX, cY = 0, 0


video = cv2.VideoCapture(0)

In [ ]:
def landmarks(imagecolor):
    landmark_list = []
    landmarkPosition = hands.process(imagecolor)
    landmark_check = landmarkPosition.multi_hand_landmarks
    if landmark_check:
        for hand in landmark_check:
            for index, landmark in enumerate(hand.landmark):
                mp_draw.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
                h, w, c = image.shape
                centerX, centerY = int(landmark.x * w), int(landmark.y * h)
                landmark_list.append([index, centerX, centerY])

    return landmark_list


def fingers(landmarks):
    fingerTips = []
    tips_id = [4, 8, 12, 16, 20]

    if landmarks[tips_id[0]][1] > landmarks[tips_id[0] - 1][1]:fingerTips.append(1)
    else: fingerTips.append(0)

    for id in range(1, 5):
        if landmarks[tips_id[id]][2] < landmarks[tips_id[id] - 3][2]:fingerTips.append(1)
        else: fingerTips.append(0)

    return fingerTips

In [ ]:

while True:
    _, image = video.read()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    landmark = landmarks(image_rgb)

    # print(landmark)

    if len(landmark) != 0:
        x1, y1 = landmark[8][1:]
        x2, y2 = landmark[12][1:]

        finger = fingers(landmark)
        print(finger)

        if finger[1] == 1 and finger[2] == 0:
            x3 = np.interp(x1, (0, 640), (0, wScr))
            y3 = np.interp(x1, (0, 640), (0, hScr))

            cX = pX + (x3 - pX) / 10
            cY = pY + (y3 - pY) / 10

            autopy.mouse.move(wScr-cX, cY)
            pX, pY = cX, cY


        if finger[0] == 1 and finger[1] == 0:
            autopy.mouse.click()



    cv2.imshow("image", image)
    if cv2.waitKey(1) == ord("o"): break



video.release()
cv2.destroyAllWindows()